# Homework 2
## Extracting TopUniversities and TimesHigherEducation Data

## Library

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Vars

In [4]:
web_urls = [{'page':'https://www.topuniversities.com',
             'data':'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051_indicators.txt'},
            {'page':'https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/25/sort_by/rank/sort_order/asc/cols/stats',
             'data':'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'}]



## Functions

In [29]:
def get_number_elements(htmlPage, className):
    try:
        number = int(((htmlPage.find('div', class_=className)).find('div', class_="number").text).replace(',',''))    
    except AttributeError:
        number = 0
    return number

def crawl_topuniversities(url,n_top):
    http_response = requests.get(url['data'])
    universities = http_response.json()['data'][:n_top]
    list_universities = []
    for idx,university in enumerate(universities):
        
        university_region = university['region']
        university_country = university['location']
        university_rank = university['overall_rank']
        
        university_info_html = BeautifulSoup(university['uni'], 'html.parser')
        link_element = university_info_html.find('a')
        
        university_name = link_element.text
        university_link = link_element.get('href')

        university_detail = requests.get(url['page']+university_link)
        university_detail_html = BeautifulSoup(university_detail.text, 'html.parser')
        
        n_f_members = get_number_elements(university_detail_html,"total faculty")    
        n_inter_f_members = get_number_elements(university_detail_html,"inter faculty") 
        
        n_students = get_number_elements(university_detail_html,"total student") 
        n_inter_students = get_number_elements(university_detail_html,"total inter") 
        
        university_data = ({
                                'Rank': university_rank,
                                'University Name': university_name,
                                'Country': university_country,
                                'Region': university_region,
                                'Total Faculty Members': n_f_members,
                                'Total International Faculty Members': n_inter_f_members,
                                'Total Students': n_students,
                                'Total International Students': n_inter_students
                          })
        list_universities.append(university_data)
    
    universities = pd.DataFrame.from_dict(list_universities)
    return universities

def crawl_timeshighereducation(url,n_top):
    http_response = requests.get(url['data'])
    universities = http_response.json()['data'][:n_top]
    list_universities = []
    for university in universities:
        
        university_rank = university['rank']
        university_name = university['name']
        university_country = university['location']
        n_students = university['stats_number_students']
        student_staff_ratio = university['stats_student_staff_ratio']
        inter_student_percentage = university['stats_pc_intl_students'] 
        
        university_data = ({
                                'Rank': university_rank,
                                'University Name': university_name,
                                'Country': university_country,
                                'Ratio Student/Faculty Members': student_staff_ratio,
                                'Percentage International Students': inter_student_percentage,
                                'Total Students': n_students
                          })
        list_universities.append(university_data)
        
    universities = pd.DataFrame.from_dict(list_universities)
    return universities   

## Main Homework

1. Analysis of Top Universities Data

In [30]:
tu_top200 = crawl_topuniversities(web_urls[0],200)

In [31]:
tu_top200.head()

,Country,Rank,Region,Total Faculty Members,Total International Faculty Members,Total International Students,Total Students,University Name
0,United States,1,North America,2982,1679,3717,11067,Massachusetts Institute of Technology (MIT)
1,United States,2,North America,4285,2042,3611,15878,Stanford University
2,United States,3,North America,4350,1311,5266,22429,Harvard University
3,United States,4,North America,953,350,647,2255,California Institute of Technology (Caltech)
4,United Kingdom,5,Europe,5490,2278,6699,18770,University of Cambridge


* Writing Data into a file

In [32]:
tu_top200.to_pickle("tu_top200_pickle")

In [ ]:
te_top200 = crawl_timeshighereducation(web_urls[1],10)

In [ ]:
te_top200.head()